In [1]:
from model.afnonet import AFNONet

In [2]:
import os 
import torch

$$
\left(
\begin{array}{ccccccc}
 \epsilon _{\text{II}}^1-\epsilon _{\text{II}}^{\text{real}} & 0 & 0 & 0 & 0 & 0 & 0 \\
 \epsilon _{\text{III}}^2-\epsilon _{\text{III}}^{\text{real}} & \epsilon _{\text{III}}^1-\epsilon _{\text{III}}^{\text{real}} & 0 & 0 & 0 & 0 & 0 \\
 \epsilon _{\text{IV}}^3-\epsilon _{\text{IV}}^{\text{real}} & \epsilon _{\text{IV}}^2-\epsilon _{\text{IV}}^{\text{real}} & \epsilon _{\text{IV}}^1-\epsilon _{\text{IV}}^{\text{real}} & 0 & 0 & 0 & 0 \\
 \epsilon _V^4-\epsilon _V^{\text{real}} & \epsilon _V^3-\epsilon _V^{\text{real}} & \epsilon _V^2-\epsilon _V^{\text{real}} & \epsilon _V^1-\epsilon _V^{\text{real}} & 0 & 0 & 0 \\
 \epsilon _{\text{VI}}^5-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^4-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^3-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^2-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^1-\epsilon _{\text{VI}}^{\text{real}} & 0 & 0 \\
 \epsilon _{\text{VII}}^6-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^5-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^4-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^3-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^2-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^1-\epsilon _{\text{VII}}^{\text{real}} & 0 \\
 \epsilon _{\text{VIII}}^7-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^6-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^5-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^4-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^3-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^2-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^1-\epsilon _{\text{VIII}}^{\text{real}} \\
\end{array}
\right)
$$

$$ 
f (X_t)+\epsilon_I^t=\hat{X_{t+1}} + \epsilon_I^t = X_{t+1}
$$

$$
f[f(X_t)] + \epsilon_{II}^t = X_{t+2}\\
f (X_{t+1})+\epsilon_I^{t+1}=\hat{X_{t+2}} + \epsilon_I^{t+1} = X_{t+2}
$$


$$
\epsilon_{II}^t = f (X_{t+1}) - f[f(X_t)]  +\epsilon_I^{t+1} = f (X_{t+1}) - f[\hat{X_{t+1}}]+\epsilon_I^{t+1}
$$

$$
\epsilon_{II}^t = \nabla f|_{X_{t+1}} [X_{t+1}-\hat{X_{t+1}}]+\epsilon_I^{t+1} =  \nabla f|_{X_{t+1}} \epsilon_I^{t} +\epsilon_I^{t+1}
$$

In [3]:
import numpy as np

In [3]:
def get_error_propagation(fourcastresult):
    error_order = torch.LongTensor([  0,
                                  2, 1,
                                  5, 4, 3,
                                  9, 8, 7, 6,
                                 14,13,12,11,10,
                                 20,19,18,17,16,15,
                                 27,26,25,24,23,22,21])
    N=7;x,y = torch.tril_indices(N,N)
    out={}
    for key in ['abs_error', 'est_error', 'Jacobianv', 'JacobV_N','abc_error']:
        b = []
        for i in range(len(fourcastresult)-10):
            a = torch.zeros(N,N)
            a[x,y] = fourcastresult[i][key][error_order]
            if key != 'JacobV_N':a=a/fourcastresult[i]['blevel_v'].mean(1)[1:,None]
            b.append(a)
        b = torch.stack(b)
        out[key] = b
    
    for key in ['Angle']:
        b = []
        for i in range(len(fourcastresult)-10):
            a = torch.zeros(N,N)
            a[x,y] = fourcastresult[i][key][error_order]
            b.append(a)
        b = torch.stack(b)
        out[key] = b
        
    for key in ['alevel_v','blevel_v','JacobVA_N']:
        b = []
        for i in range(len(fourcastresult)-10):
            b.append(fourcastresult[i][key])
        b = torch.stack(b)
        out[key] = b
    return out

In [5]:
def get_fourcastresult(fourcastresult):
    #fourcastresult="checkpoints/WeathBench7066/AFNONet/ts_2_pretrain-2D706N_per_1_step/01_06_12_22-seed_73001"
    ROOT= fourcastresult
    fourcastresult_list = [os.path.join(ROOT,p) for p in os.listdir(fourcastresult) if 'fourcastresult.gpu' in p]
    fourcastresult={}
    for save_path in fourcastresult_list:
        tmp = torch.load(save_path)
        for key,val in tmp.items():
            if key not in fourcastresult:
                fourcastresult[key] = val
            else:
                if key == 'global_rmse_map':

                    fourcastresult['global_rmse_map'] = [a+b for a,b in zip(fourcastresult['global_rmse_map'],tmp['global_rmse_map'])]
                else:
                    fourcastresult[key] = val # overwrite
    return fourcastresult

In [7]:
fourcastresult_pretrain="checkpoints/WeathBench7066/AFNONet/ts_2_pretrain-2D706N_per_1_step/01_06_12_22-seed_73001"
fourcastresult_pretrain = get_fourcastresult(fourcastresult_pretrain)
# torch.save(fourcastresult_pretrain,fourcastresult+"/fourcastresult.mid_tangent.pt")

In [8]:
fourcastresult_finetune="checkpoints/WeathBench7066/AFNONet/ts_3_finetune-2D706N_per_1_step/01_09_02_41_31-seed_73001"
fourcastresult_finetune = get_fourcastresult(fourcastresult_finetune)
#torch.save(fourcastresult_finetune,fourcastresult+"/fourcastresult.right_tangent.pt")

In [130]:
fourcastresult_pretrain1 = torch.load(fourcastresult_pretrain+"/fourcastresult.right_tangent.pt")
fourcastresult_pretrain2 = torch.load(fourcastresult_pretrain+"/fourcastresult.mid_tangent.pt")
fourcastresult_finetune1 = torch.load(fourcastresult_finetune+"/fourcastresult.right_tangent.pt")
fourcastresult_finetune2 = torch.load(fourcastresult_finetune+"/fourcastresult.mid_tangent.pt")

In [6]:
import torch

In [7]:
from mltool.visualization import *

In [47]:
_component_list= ([58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,  1]+ # u component of wind and the 10m u wind
                    [71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,  2]+   # v component of wind and the 10m v wind
                    [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  0]+   # Temperature and the 2m_temperature
                    [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18]+   # Geopotential and the last one is ground Geopotential, should be replace later
                    [45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 57]    # Realitve humidity and the Realitve humidity at groud, should be modified by total precipitaiton later
                    )
single_vnames = ["2m_temperature",
                  "10m_u_component_of_wind",
                  "10m_v_component_of_wind",
                  "total_cloud_cover",
                  "total_precipitation",
                  "toa_incident_solar_radiation"]
level_vnames= []
for physics_name in ["geopotential", "temperature",
                     "specific_humidity","relative_humidity",
                     "u_component_of_wind","v_component_of_wind",
                     "vorticity","potential_vorticity"]:
    for pressure_level in [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]:
        level_vnames.append(f"{pressure_level}hPa_{physics_name}")
all_vnames = single_vnames + level_vnames
vnames= np.array([all_vnames[i] for i in _component_list])

In [141]:
# %matplotlib inline
# fig = plt.figure(figsize=(12,24))
# _=plt.imshow(fourcastresult_finetune[5]['blevel_v'].numpy())
# _=plt.xticks(range(70),vnames,rotation=-90)

In [8]:
fourcastresult_finetune[0].keys()

dict_keys(['accu', 'rmse', 'std_pred', 'std_true', 'snap_line', 'hmse', 'abs_error', 'est_error', 'abc_error', 'alevel_v', 'blevel_v', 'Jacobianv', 'JacobV_N', 'JacobVA_N', 'Angle'])

In [63]:
D3error = fourcastresult_finetune[5]['blevel_v'].reshape(8,5,14).numpy()
D3name  = vnames.reshape(5,14)

In [157]:
# property_index = 3
# data = D3error[:,property_index]
# name = D3name[property_index]
# fig = plt.figure(figsize=(12,24))
# _=plt.imshow(data)
# _=plt.xticks(range(len(name)),name,rotation=-90)

In [10]:
error_propagation_finetune = get_error_propagation(fourcastresult_finetune)

In [11]:
error_propagation_finetune['abs_error'].mean(0)

tensor([[0.0014, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0038, 0.0031, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0079, 0.0073, 0.0057, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0143, 0.0138, 0.0125, 0.0094, 0.0000, 0.0000, 0.0000],
        [0.0246, 0.0241, 0.0229, 0.0205, 0.0147, 0.0000, 0.0000],
        [0.0381, 0.0375, 0.0364, 0.0342, 0.0296, 0.0207, 0.0000],
        [0.0554, 0.0548, 0.0536, 0.0514, 0.0475, 0.0406, 0.0273]])

In [13]:
error_propagation_finetune['est_error'].mean(0)

tensor([[-0.0650,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0670, -0.0633,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0797, -0.0773, -0.0690,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0960, -0.0941, -0.0887, -0.0746,  0.0000,  0.0000,  0.0000],
        [-0.1249, -0.1231, -0.1186, -0.1094, -0.0878,  0.0000,  0.0000],
        [-0.1592, -0.1574, -0.1534, -0.1459, -0.1314, -0.1011,  0.0000],
        [-0.2002, -0.1983, -0.1944, -0.1876, -0.1760, -0.1555, -0.1154]])

In [14]:
error_propagation_finetune['abc_error'].mean(0)

tensor([[0.0084, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0212, 0.0177, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0394, 0.0372, 0.0292, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0632, 0.0613, 0.0560, 0.0425, 0.0000, 0.0000, 0.0000],
        [0.0945, 0.0927, 0.0883, 0.0792, 0.0583, 0.0000, 0.0000],
        [0.1312, 0.1294, 0.1255, 0.1181, 0.1038, 0.0743, 0.0000],
        [0.1740, 0.1722, 0.1683, 0.1616, 0.1500, 0.1299, 0.0908]])

In [233]:
error_propagation_pretrain['blevel_v'].mean(dim=(0,2))

tensor([0.0332, 0.0471, 0.0581, 0.0686, 0.0796, 0.0912, 0.1036, 0.1169])

In [202]:
(np.arccos(error_propagation_pretrain['Angle'])/np.pi*180).mean(0)

tensor([[90.4942, 90.0000, 90.0000, 90.0000, 90.0000, 90.0000, 90.0000],
        [90.1774, 90.1678, 90.0000, 90.0000, 90.0000, 90.0000, 90.0000],
        [90.1830, 90.1806, 90.1576, 90.0000, 90.0000, 90.0000, 90.0000],
        [90.0862, 90.0853, 90.0784, 90.0434, 90.0000, 90.0000, 90.0000],
        [90.1888, 90.1879, 90.1847, 90.1731, 90.1292, 90.0000, 90.0000],
        [90.2654, 90.2649, 90.2627, 90.2587, 90.2444, 90.1946, 90.0000],
        [90.3459, 90.3449, 90.3436, 90.3394, 90.3327, 90.3141, 90.2579]])

In [131]:
error_propagation_pretrain1 = get_error_propagation(fourcastresult_pretrain1)
error_propagation_pretrain2 = get_error_propagation(fourcastresult_pretrain2)

In [87]:
error_propagation_finetune1 = get_error_propagation(fourcastresult_finetune1)
error_propagation_finetune2 = get_error_propagation(fourcastresult_finetune2)

In [88]:
error_propagation_finetune1['est_error'].mean(0)

tensor([[0.2625, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2788, 0.2437, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3135, 0.2873, 0.2431, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3501, 0.3301, 0.2978, 0.2413, 0.0000, 0.0000, 0.0000],
        [0.3982, 0.3820, 0.3581, 0.3196, 0.2506, 0.0000, 0.0000],
        [0.4518, 0.4379, 0.4185, 0.3906, 0.3455, 0.2636, 0.0000],
        [0.5103, 0.4979, 0.4815, 0.4592, 0.4272, 0.3756, 0.2796]])

In [89]:
error_propagation_finetune2['est_error'].mean(0)

tensor([[0.2532, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2569, 0.2496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2804, 0.2762, 0.2607, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3079, 0.3048, 0.2959, 0.2712, 0.0000, 0.0000, 0.0000],
        [0.3511, 0.3485, 0.3422, 0.3285, 0.2941, 0.0000, 0.0000],
        [0.3966, 0.3944, 0.3892, 0.3796, 0.3603, 0.3159, 0.0000],
        [0.4449, 0.4427, 0.4383, 0.4306, 0.4170, 0.3920, 0.3376]])

In [91]:
error_propagation_finetune1['abs_error'].mean(0)

tensor([[0.1744, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2636, 0.2270, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3458, 0.3258, 0.2731, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4255, 0.4116, 0.3828, 0.3155, 0.0000, 0.0000, 0.0000],
        [0.5055, 0.4944, 0.4744, 0.4370, 0.3552, 0.0000, 0.0000],
        [0.5869, 0.5773, 0.5614, 0.5351, 0.4890, 0.3919, 0.0000],
        [0.6687, 0.6600, 0.6464, 0.6256, 0.5933, 0.5380, 0.4252]])

In [105]:
error_propagation_finetune2['abs_error'].mean(0)

tensor([[0.0366, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0600, 0.0541, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0868, 0.0837, 0.0738, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1174, 0.1153, 0.1099, 0.0952, 0.0000, 0.0000, 0.0000],
        [0.1534, 0.1518, 0.1480, 0.1397, 0.1187, 0.0000, 0.0000],
        [0.1914, 0.1900, 0.1869, 0.1811, 0.1690, 0.1412, 0.0000],
        [0.2313, 0.2300, 0.2274, 0.2227, 0.2142, 0.1979, 0.1623]])

In [132]:
error_propagation_pretrain1['abs_error'].mean(0)

tensor([[0.1748, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2731, 0.2385, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3630, 0.3448, 0.2902, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4492, 0.4370, 0.4078, 0.3359, 0.0000, 0.0000, 0.0000],
        [0.5342, 0.5246, 0.5048, 0.4656, 0.3768, 0.0000, 0.0000],
        [0.6194, 0.6114, 0.5960, 0.5694, 0.5200, 0.4136, 0.0000],
        [0.7047, 0.6975, 0.6848, 0.6641, 0.6304, 0.5699, 0.4461]])

In [133]:
error_propagation_pretrain2['abs_error'].mean(0)

tensor([[0.0368, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0649, 0.0592, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0975, 0.0946, 0.0833, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1345, 0.1326, 0.1266, 0.1090, 0.0000, 0.0000, 0.0000],
        [0.1747, 0.1732, 0.1690, 0.1591, 0.1338, 0.0000, 0.0000],
        [0.2180, 0.2166, 0.2133, 0.2063, 0.1915, 0.1580, 0.0000],
        [0.2640, 0.2628, 0.2599, 0.2542, 0.2438, 0.2238, 0.1809]])

In [135]:
error_propagation_finetune1['JacobV_N'].mean(0).sqrt()

tensor([[0.8029, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8482, 0.8438, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8827, 0.8805, 0.8755, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9094, 0.9080, 0.9060, 0.9005, 0.0000, 0.0000, 0.0000],
        [0.9315, 0.9304, 0.9291, 0.9276, 0.9214, 0.0000, 0.0000],
        [0.9500, 0.9491, 0.9480, 0.9470, 0.9459, 0.9389, 0.0000],
        [0.9654, 0.9646, 0.9638, 0.9629, 0.9623, 0.9619, 0.9539]])

In [136]:
error_propagation_finetune2['JacobV_N'].mean(0).sqrt()

tensor([[0.7998, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8452, 0.8451, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8789, 0.8787, 0.8791, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9054, 0.9052, 0.9049, 0.9062, 0.0000, 0.0000, 0.0000],
        [0.9279, 0.9277, 0.9274, 0.9273, 0.9295, 0.0000, 0.0000],
        [0.9456, 0.9454, 0.9451, 0.9448, 0.9450, 0.9487, 0.0000],
        [0.9601, 0.9600, 0.9597, 0.9594, 0.9592, 0.9598, 0.9649]])

In [139]:
print(error_propagation_pretrain1['JacobVA_N'].mean(0).sqrt())
print(error_propagation_pretrain2['JacobVA_N'].mean(0).sqrt())
print(error_propagation_finetune1['JacobVA_N'].mean(0).sqrt())
print(error_propagation_finetune2['JacobVA_N'].mean(0).sqrt())

tensor([0.6604, 0.6604, 0.6604, 0.6604, 0.6604, 0.6604, 0.6604])
tensor([0.6671, 0.6671, 0.6669, 0.6669, 0.6668, 0.6667, 0.6667])
tensor([0.6448, 0.6448, 0.6448, 0.6449, 0.6449, 0.6449, 0.6449])
tensor([0.6399, 0.6407, 0.6404, 0.6403, 0.6404, 0.6403, 0.6402])


In [96]:
error_propagation_finetune2['JacobVA_N'].mean(0).sqrt()

tensor([0.6399, 0.6407, 0.6404, 0.6403, 0.6404, 0.6403, 0.6402])

In [188]:
error_propagation_pretrain['alevel_v'].mean(0).sqrt()/error_propagation_finetune['blevel_v'].mean(0).sqrt()

tensor([0.8785, 0.8052, 0.7498, 0.7022, 0.6603, 0.6228, 0.5887])

In [182]:
error_propagation_pretrain['Jacobianv'].mean(0)

tensor([[0.5542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6753, 0.6637, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7563, 0.7471, 0.7312, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8206, 0.8132, 0.8006, 0.7801, 0.0000, 0.0000, 0.0000],
        [0.8786, 0.8722, 0.8621, 0.8462, 0.8201, 0.0000, 0.0000],
        [0.9336, 0.9279, 0.9191, 0.9064, 0.8863, 0.8535, 0.0000],
        [0.9874, 0.9821, 0.9743, 0.9632, 0.9473, 0.9220, 0.8816]])

In [79]:
error_propagation_finetune['est_error'].mean(0)

tensor([[0.2532, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2569, 0.2496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2804, 0.2762, 0.2607, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3079, 0.3048, 0.2959, 0.2712, 0.0000, 0.0000, 0.0000],
        [0.3511, 0.3485, 0.3422, 0.3285, 0.2941, 0.0000, 0.0000],
        [0.3966, 0.3944, 0.3892, 0.3796, 0.3603, 0.3159, 0.0000],
        [0.4449, 0.4427, 0.4383, 0.4306, 0.4170, 0.3920, 0.3376]])

In [189]:
error_propagation_finetune['Jacobianv'].mean(0)

tensor([[0.5603, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6665, 0.6527, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7407, 0.7300, 0.7138, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8007, 0.7922, 0.7793, 0.7596, 0.0000, 0.0000, 0.0000],
        [0.8561, 0.8487, 0.8382, 0.8225, 0.7983, 0.0000, 0.0000],
        [0.9089, 0.9020, 0.8928, 0.8800, 0.8609, 0.8315, 0.0000],
        [0.9606, 0.9541, 0.9456, 0.9344, 0.9190, 0.8961, 0.8603]])

In [199]:
error_propagation_finetune['est_error'].mean(0)*error_propagation_finetune['blevel_v'].mean(0)[:,None].sqrt()/error_propagation_pretrain['blevel_v'].mean(0)[:,None].sqrt()

tensor([[0.2508, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2616, 0.2287, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2907, 0.2664, 0.2254, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3219, 0.3035, 0.2738, 0.2218, 0.0000, 0.0000, 0.0000],
        [0.3637, 0.3488, 0.3270, 0.2919, 0.2289, 0.0000, 0.0000],
        [0.4104, 0.3978, 0.3802, 0.3548, 0.3138, 0.2395, 0.0000],
        [0.4618, 0.4506, 0.4357, 0.4156, 0.3866, 0.3398, 0.2530]])

In [192]:
error_propagation_pretrain['blevel_v'].mean(0).sqrt()

tensor([0.2170, 0.2410, 0.2619, 0.2821, 0.3020, 0.3219, 0.3418])

In [191]:
error_propagation_finetune['blevel_v'].mean(0).sqrt()

tensor([0.2073, 0.2262, 0.2429, 0.2593, 0.2758, 0.2924, 0.3093])

In [171]:
x, y  = np.meshgrid(np.arange(7),np.arange(7))
z = error_propagation_finetune['abs_error'].mean(0).numpy()
data = np.stack([x,y,z],-1).reshape(-1,3)
data = data[data[:,2]>0]

In [204]:
weight = torch.load("checkpoints/WeathBench7066/AFNONet/time_step_2_pretrain-2D706N_every_1_step/10_14_20_18_11-seed_73001/backbone.best.pt", map_location='cpu')

In [206]:
[k for k in weight['model'] if 'patch_embed.proj' in k]

['patch_embed.proj.conv_layers.0.weight',
 'patch_embed.proj.conv_layers.1.weight']

In [70]:
error_propagation_finetune['JacobV_N'].mean(0)

tensor([[0.6446, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7194, 0.7120, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7791, 0.7752, 0.7665, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8270, 0.8244, 0.8209, 0.8109, 0.0000, 0.0000, 0.0000],
        [0.8678, 0.8656, 0.8632, 0.8604, 0.8489, 0.0000, 0.0000],
        [0.9025, 0.9008, 0.8987, 0.8968, 0.8948, 0.8815, 0.0000],
        [0.9320, 0.9305, 0.9289, 0.9272, 0.9260, 0.9253, 0.9100]])

In [72]:
error_propagation_pretrain['Jacobianv'].mean(0)

tensor([[0.5542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6753, 0.6637, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7563, 0.7471, 0.7312, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8206, 0.8132, 0.8006, 0.7801, 0.0000, 0.0000, 0.0000],
        [0.8786, 0.8722, 0.8621, 0.8462, 0.8201, 0.0000, 0.0000],
        [0.9336, 0.9279, 0.9191, 0.9064, 0.8863, 0.8535, 0.0000],
        [0.9874, 0.9821, 0.9743, 0.9632, 0.9473, 0.9220, 0.8816]])

In [35]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")

In [42]:
class NGmod_RotationDeltaX(Nodal_GradientModifier):
    def normed(self,a):
        shape = a.shape
        a = a.reshape(a.size(0),-1)
        a = a/a.norm(dim=1,keepdim=True)
        a = a.reshape(shape)
        return a 

    def getRotationDeltaloss(self, modelfun, x,  t , rotation_regular_mode = '0y0'):
        y = model(x)
        if rotation_regular_mode =='0y0':
            delta = (self.normed(y - x),)
        elif rotation_regular_mode =='0v0':
            delta = (self.normed(y.detach() - x),)
        elif rotation_regular_mode =='Yy0':
            delta = (self.normed(t - x),self.normed(y - x))
        elif rotation_regular_mode =='Yv0':
            delta = (self.normed(t - x),self.normed(y.detach() - x))
        elif rotation_regular_mode =='YyN':
            delta = (self.normed(t - x),self.normed(y - x),self.normed(torch.rand_like(x)))
        else:
            raise NotImplementedError
        penalty= 0
        for delta_cons in delta:
            grad = functorch.jvp(modelfun, (x,), (delta_cons,))[1] 
            position_range = list(range(1,len(grad.shape))) # (B,P, W,H ) --> (1,2,3)
            penalty       += ((torch.sum(grad**2,dim=position_range).sqrt()-1)**2).mean()
        return penalty

    def getL2loss(self,modelfun,x,chunk=10,coef=None):
        raise
    def getL1loss(self,modelfun,x,chunk=10,coef=None):
        raise

In [8]:
from einops.layers.torch import Rearrange

In [9]:
import torch.nn as nn

In [10]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(30,30)
    def forward(self,x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        x = x**2
        return x
model = MyModel()
torch.save(model.state_dict(),"debug/testmodel2.pt")
x = torch.randn(2,30)
y = torch.randn(2,30)
torch.save(x,"debug/testx2.pt")
torch.save(y,"debug/testy2.pt")

In [11]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")
ngmod = NGmod_RotationDeltaX(0,0,0 )

In [16]:
cotangents1 = torch.randn(3,*x.shape)

In [43]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")
ngmod = NGmod_RotationDeltaX(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x, y,rotation_regular_mode='YyN')
print(loss)
loss.backward()
ll = [[name,p.grad.norm()] for idx,(name,p) in enumerate(model.named_parameters()) if p.grad is not None]
for i in range(-10,-1,1):
    name,pnorm = ll[i] 
    print(f"{name} - {pnorm}")
    

tensor(1.8809, grad_fn=<AddBackward0>)
blocks.11.mlp.fc1.weight - 0.12911739945411682
blocks.11.mlp.fc1.bias - 0.003467740025371313
norm.weight - 0.028279943391680717
norm.bias - 0.0017165266908705235
pre_logits.conv1.weight - 2.469726085662842
pre_logits.conv1.bias - 0.0033190883696079254
pre_logits.conv2.weight - 1.0886025428771973
pre_logits.conv2.bias - 0.002785270567983389
head.weight - 0.2757563292980194


In [91]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")
ngmod = NGmod_RotationDelta2(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x, model(x), y,rotation_regular_mode='0y0')
print(loss)
loss.backward()
ll = [[name,p.norm()] for idx,(name,p) in enumerate(model.named_parameters()) if p.grad is not None]
for i in range(-10,-1,1):
    name,pnorm = ll[i] 
    print(f"{name} - {pnorm}")
    

tensor(0.9506, grad_fn=<MeanBackward0>)
blocks.11.mlp.fc1.weight - 30.728208541870117
blocks.11.mlp.fc1.bias - 0.0
norm.weight - 27.712812423706055
norm.bias - 0.0
pre_logits.conv1.weight - 15.994429588317871
pre_logits.conv1.bias - 0.19202826917171478
pre_logits.conv2.weight - 19.3148250579834
pre_logits.conv2.bias - 0.1864301562309265
head.weight - 9.63589096069336


In [82]:
model = MyModel()
model.load_state_dict(torch.load("debug/testmodel2.pt"))
x = torch.load("debug/testx2.pt")
y = torch.load("debug/testy2.pt")
ngmod = NGmod_RotationDelta(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x,model(x),y,rotation_regular_mode='0y0')
print(loss)
loss.backward()
for idx,(name,p) in enumerate(model.named_parameters()):
    print(f"{name} - {p.grad.norm()}")
    

tensor(0.0157, grad_fn=<MeanBackward0>)
linear.weight - 0.047377560287714005
linear.bias - 0.004780657589435577


In [81]:
model = MyModel()
model.load_state_dict(torch.load("debug/testmodel2.pt"))
x = torch.load("debug/testx2.pt")
y = torch.load("debug/testy2.pt")
ngmod = NGmod_RotationDelta2(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x,model(x),y,rotation_regular_mode='0y0')
print(loss)
loss.backward()
for idx,(name,p) in enumerate(model.named_parameters()):
    print(f"{name} - {p.grad.norm()}")
    

tensor(0.9756, grad_fn=<MeanBackward0>)
linear.weight - 0.0752129852771759
linear.bias - 0.005585702136158943


In [78]:
class NGmod_RotationDelta2(Nodal_GradientModifier):
    def normed(self,a):
        shape = a.shape
        a = a.reshape(a.size(0),-1)
        a = a/a.norm(dim=1,keepdim=True)
        a = a.reshape(shape)
        return a 

    def getRotationDeltaloss(self, modelfun, x, y, t , rotation_regular_mode = '0y0'):
        #y, vjpfunc = functorch.vjp(modelfun, x) # notice this will calculate f(x) again, so can be reduced in real implement.
        if rotation_regular_mode =='0y0':
            delta = self.normed(y - x)
        elif rotation_regular_mode =='0v0':
            delta = self.normed(y.detach() - x)
        elif rotation_regular_mode =='Yy0':
            delta = torch.cat([self.normed(t - x),self.normed(y - x)])
        elif rotation_regular_mode =='Yv0':
            delta = torch.cat([self.normed(t - x),self.normed(y.detach() - x)])
        elif rotation_regular_mode =='YyN':
            delta = torch.cat([self.normed(t - x),self.normed(y - x),self.normed(torch.rand_like(x))])
        else:
            raise NotImplementedError
        #_,vjpfunc = functorch.vjp(modelfun, x) # notice this will calculate f(x) again, so can be reduced in real implement.    
        #grad = vjpfunc(delta)[0]
        grad           = functorch.jvp(model, (x,), (delta,))[1]
        position_range = list(range(1,len(grad.shape))) # (B,P, W,H ) --> (1,2,3)
        penalty        = ((torch.sum(grad**2,dim=position_range)-1)**2).mean()
        return penalty